# 9章

- 本章で用いる「[日本語Wikipedia入力誤りデータセット](https://nlp.ist.i.kyoto-u.ac.jp/?%E6%97%A5%E6%9C%AC%E8%AA%9EWikipedia%E5%85%A5%E5%8A%9B%E8%AA%A4%E3%82%8A%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88)」は現在バージョン2が公開されていますが、本章では[バージョン1](https://nlp.ist.i.kyoto-u.ac.jp/?%E6%97%A5%E6%9C%AC%E8%AA%9EWikipedia%E5%85%A5%E5%8A%9B%E8%AA%A4%E3%82%8A%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88v1)を用いています。

本章では、文章校正タスクのうち漢字の誤変換タスクを扱う。BERTでは、文章校正をトークンの分類問題として扱うことで実装できる。<br>
正しいトークンについては同じトークンを、間違っていると考えられるトークンにはBERTの語彙の中から正しいと予測したトークンを返す。

|誤変換|正しい文章|
|:-|:-|
|優勝|優勝|
|トロフィー|トロフィー|
|を|を|
|変換|返還|
|し|し|
|た|た|
|。|。|

一方で、以下のようにこの方法では扱えない文章が存在する。

|誤変換|正しい文章|
|:-|:-|
|投|当初|
|##書|は|
|は|、|
|、|実行|
|実行|を|

本章では、上記のような文章を取り扱わないこととする。

In [ ]:
# 9-3

import os
import random
import unicodedata
import pandas as pd
from tqdm import tqdm
import pprint
import math

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pytorch_lightning as pl

トークナイザを定義する。

In [ ]:
# 9-4

class SC_tokenizer(BertJapaneseTokenizer):
    def encode_plus_tagged(
        self,
        wrong_text: str,
        correct_text: str,
        max_length: int = 128,
    ) -> dict:
        """
        ファインチューニング用
        誤変換を含む文章と正しい文章を渡し符号化し、
        誤変換文章のlabelsを正しい文章のinput_idsに置き換える
        """
        # 文章から直接符号化
        # tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)で
        # インスタンス化したtokinzer(text)関数と同等
        wrong_encoding = self(
            wrong_text, max_length=max_length, padding='max_length', truncation=True
        )
        correct_encoding = self(
            correct_text, max_length=max_length, padding='max_length', truncation=True
        )
        # 正しい文章の符号をラベルとする
        wrong_encoding['labels'] = correct_encoding['input_ids']

        return wrong_encoding
    
    def encode_plus_untagged(
        self,
        text: str,
        max_length: int = None,
        return_tensors: str = 'pt',
    ) -> (dict, list):
        """
        文章をトークン化し、それぞれのトークンと文章中の文字列を対応付ける
        推論時にトークンごとのラベルを予測し、最終的に固有表現に変換する
        未知語や文章中の空白(MeCabにより消去される)に対しての処理が必要となる
        そのため、各トークンが元の文章のどの位置にあったかを特定しておく
        """
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        tokens = []
        tokens_original = []
        # 単語をサブワードに分割してlistに格納
        for word in words:
            tokens_subword = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_subword)
            # 未知語対応
            if tokens_subword[0] == '[UNK]':
                tokens_original.append(word)
            else:
                tokens_original.extend([token.replace("##", "") for token in tokens_subword])
        
        # トークンが文章中のどの位置にあるかを走査する
        position = 0
        spans = []
        for token in tokens_original:
            length = len(token)
            while True:
                if token != text[position: position + length]:
                    position += 1
                else:
                    spans.append([position, position + length])
                    position += length
                    break
        
        # トークンをID化する
        input_ids = self.convert_tokens_to_ids(tokens)
        # トークンIDを符号化する
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length,
            padding = 'max_length' if max_length else False,
            truncation = True if max_length else False,
        )
        sequence_length = len(encoding['input_ids']) # 符号化した文章の長さ
        # 先頭トークン[CLS]用のspanを追加する
        # このとき、次の[SEP]トークンを一緒に削除しておく
        spans = [[-1, -1]] + spans[:sequence_length - 2]
        # 末尾トークン[SEP]、末尾の空トークン[PAD]用のspanを追加する
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # 引数に応じてtorch.Tensor型に返還
        # 次元を追加する必要がある
        if return_tensors == 'pt':
            encoding = {key: torch.tensor([value]) for key, value in encoding.items()}
        
        return (encoding, spans)
    
    def convert_output_to_text(
        self, 
        text: str,
        labels_arg: list,
        spans_arg: list,
    ) -> str:
        """
        文章、各トークンのラベルの予測値、文章中での位置から
        予測された文章に変換する
        """
        # 文章の長さチェック
        assert len(labels_arg) == len(spans_arg)

        # 特殊トークンを取り除く
        labels = []
        spans = []
        for label, span in zip(labels_arg, spans_arg):
            if span[0] != -1:
                labels.append(label)
                spans.append(span)
        
        # モデルが予測した文章を生成する
        text_pred = ""
        position = 0
        for label, span in zip(labels, spans):
            start, end = span
            # 空白文字の処理
            if position != start:
                text_pred += text[position: start]
            token_pred = self.convert_ids_to_tokens(label) # labelをトークン化
            token_pred = token_pred.replace("##", "") # サブワードの##を削除
            token_pred = unicodedata.normalize('NFKC', token_pred) # 文字列の正規化
            text_pred += token_pred
            position = end
        return text_pred

定義したトークナイザの動きを確認する。

モデルとトークナイザの呼び出し

In [ ]:
# 9-5

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = SC_tokenizer.from_pretrained(model_name)

`encode_plus_tagged()`メソッドは、誤変換した文章と正しい文章が渡されたら、誤変換した文章を符号化し`labels`に正しい文章のトークンIDを付与する。

In [ ]:
# 9-6

wrong_text = "優勝トロフィーを変換した"
correct_text = "優勝トロフィーを返還した"

encoding = tokenizer.encode_plus_tagged(
    wrong_text, correct_text, max_length=12,
)

In [ ]:
pprint.pprint(encoding)

`encode_plus_untagged()`メソッドは、文章を符号化し、空白や未知語を考慮した上でそれぞれのトークンの位置を返す。

In [ ]:
# 9-7

wrong_text = "優勝トロフィーを変換した"
encoding, spans = tokenizer.encode_plus_untagged(
    wrong_text, return_tensors='pt'
)

In [ ]:
encoding

In [ ]:
spans

`convert_output_to_text()`関数は、文章とラベル列、各トークンの文章中の位置から予測した文章を出力する。

In [ ]:
# 9-8

labels_pred = [2, 759, 18204, 11, 8274, 15, 10, 3]
text_pred = tokenizer.convert_output_to_text(
    wrong_text, labels_pred, spans
)

In [ ]:
text_pred

## 9.2 BERTにおける実装

`transformers.BertForMaskedLM`クラスを用いる。このクラスは各トークンに対してその市に入るトークンを語彙の中から選ぶもので、これは`transformers.BertForTokenClassification`でラベル数を語彙数としたものと入出力関係が同じと考えることができる。<br>
一方で、`transformers.BertForMaskedLM`は、事前学習でランダムに選ばれたトークンを`[MASK]`もしくはランダムなトークンに置き換え、またはそのままで、元のトークンが何であったかを予測するという学習が行われる。これは、文章校正タスクと類似しており、分類器の初期パラメータとして事前学習により得られたものを用いる`transformers.BertForMaskedLM`のほうが、`transformers.BertForTokenClassification`と比較してある程度妥当なパラメータを備えていることが期待されるためである。これにより、ファインチューニングの学習時間の短縮が期待できる。

In [ ]:
# 9-9

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
model = BertForMaskedLM.from_pretrained(model_name)
model = model.cuda()

文章の符号化から予測された文章の出力までは、以下のように実装する。

In [ ]:
# 9-10

text = "優勝トロフィーを変換した。"

# 符号化およびトークンの文章中の位置を取得
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
encoding = {key: value.cuda() for key, value in encoding.items()}

# BERTに入力
with torch.no_grad():
    output = model(**encoding)
scores = output.logits
labels_pred = scores[0].argmax(axis=1).cpu().numpy().tolist()

text_pred = tokenizer.convert_output_to_text(
    text, labels_pred, spans
)

In [ ]:
text_pred

次に、誤変換した文章と正しい文章をBERTに入力して損失を計算する。入力時に、それぞれの文章を符号化し、正しい文章についてはラベル列を正とする。`tokenizer.encode_plus_tagged()`を用いることで、ラベル列を含んだデータを作成できる。

In [ ]:
# 9-11

data = [
    {
        'wrong_text': '優勝トロフィーを変換した。',
        'correct_text': '優勝トロフィーを返還した。',
    },
    {
        'wrong_text': '人と森は強制している。',
        'correct_text': '人と森は共生している。',
    }
]

# データの符号化
max_length = 32
dataset_for_loader = []
for sample in data:
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    encoding = tokenizer.encode_plus_tagged(
        wrong_text, correct_text, max_length=max_length
    )
    encoding = {key: torch.tensor(value) for key, value in encoding.items()}
    dataset_for_loader.append(encoding)

dataloader = DataLoader(dataset_for_loader, batch_size=2)

In [ ]:
for batch in dataloader:
    encoding = {key: value.cuda() for key, value in batch.items()}
    output = model(**encoding)
    loss = output.loss

In [ ]:
loss

## 9.4 日本語Wikipedia誤りデータセット

本章では、京都大学の言語メディア研究室が作成した[日本語Wikipedia誤りデータセット]("https://nlp.ist.i.kyoto-u.ac.jp/?日本語Wikipedia入力誤りデータセット")を用いて学習を行う。このデータセットは、日本語Wikipediaの差分から間違った文章と正しい文章のペアを抽出している。

まずは、データセット(v2.0)をダウンロードする。

In [ ]:
# 9-2

os.makedirs("data/chapter9", exist_ok=True)

In [ ]:
# 9-12

!curl -o "data/chapter9/jwtd.tar.gz" "https://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JWTD/jwtd_v2.0.tar.gz&name=JWTDv2.0.tar.gz"
!cd "data/chapter9" && tar -zxf "jwtd.tar.gz"

本データセットはJSON形式で、`pre_text`に修正前の文章、`post_text`に修正後の文章が格納されている。
さらに、`diffs`配列の中の`category`に入力誤りの種類(誤字、脱字など)が、`pre_str`が修正前、`post_str`が修正後の単語が格納されている。

本章では漢字誤変換のみを扱う。`category`が`kanji-conversion_a`となっているもののみを利用する。

まずデータを読み込む。

In [ ]:
# 9-13

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = SC_tokenizer.from_pretrained(model_name)

In [ ]:
category_type = 'kanji-conversion_a'

def create_dataset(data_df: pd.DataFrame) -> dict:
    def _check_category(lst: list) -> bool:
        """
        data_df.diffsを走査して、
        - listの長さが2以下
        - すべてのcategoryがkanji-conversion_a
        のすべてを満たすときにTrue、そうでないときにFalseを返す
        """
        checked_lst = [dct['category'] == category_type for dct in lst]
        category_bool =  math.prod(checked_lst) # リスト内を掛け算してFalseがあれば0が返る
        length_bool = len(checked_lst) <= 2
        return bool(category_bool * length_bool)

    def _normalize(text: str) -> str:
        """
        文字列の正規化を行う
        """
        text = text.strip() # 改行文字や全角スペースを取り除く
        return unicodedata.normalize('NFKC', text) # NFKCで正規化

    def check_token_count(row):
        pre_text_tokens = tokenizer.tokenize(row['pre_text'])
        post_text_tokens = tokenizer.tokenize(row['post_text'])
        if len(pre_text_tokens) != len(post_text_tokens):
            return False
        
        diff_count = 0
        threshold_count = 2
        for pre_text_token, post_text_token in zip(pre_text_tokens, post_text_tokens):
            if pre_text_token != post_text_token:
                diff_count += 1
                if diff_count > threshold_count:
                    return False
        else:
            return True

    # data_df.diffsのtypeをobject -> listに変換
    data_df.diffs = data_df.diffs.apply(lambda x: list(x))

    # data_dfからcategoryがkanji-conversion_aかつ誤変換が2以下の文章のみを抜き出す
    data_df = data_df[data_df['diffs'].apply(_check_category)].copy()

    # data_dfからトークンの差が以下の文章のみを抜き出す
    data_df = data_df[data_df.apply(check_token_count, axis=1)].copy()

    # 文章の正規化
    data_df.pre_text = data_df.pre_text.apply(_normalize)
    data_df.post_text = data_df.post_text.apply(_normalize)

    return data_df[['pre_text', 'post_text']].to_dict(orient='records')

In [ ]:
# データセットを読み込んで加工する
dir_path = "data/chapter9/jwtd_v2.0/"
train_df = pd.read_json(dir_path+"train.jsonl", orient='records', lines=True)
test_df = pd.read_json(dir_path+"test.jsonl", orient='records', lines=True)

# train/val
tmp_dataset = create_dataset(train_df)
random.shuffle(tmp_dataset)
train_size = int(len(tmp_dataset) * 0.8)
train_dataset = tmp_dataset[:train_size]
val_dataset = tmp_dataset[train_size:]
# test
test_dataset = create_dataset(test_df)

## 9.5 ファインチューニング

ファインチューニングのためのデータローダを定義する。

In [ ]:
# 9-14

def create_dataset_for_loader(
        tokenizer,
        dataset: dict,
        max_length: int
    ):
    """
    データセットをデータローダに入力可能な形式にする
    """
    dataset_for_loader = []
    for sample in tqdm(dataset):
        pre_text = sample['pre_text']
        post_text = sample['post_text']
        encoding = tokenizer.encode_plus_tagged(
            pre_text, post_text, max_length=max_length
        )
        encoding = {key: torch.tensor(value) for key, value in encoding.items()}
        dataset_for_loader.append(encoding)
    return dataset_for_loader


max_length = 32
train_dataset_for_loader = create_dataset_for_loader(
    tokenizer, train_dataset, max_length
)
val_dataset_for_loader = create_dataset_for_loader(
    tokenizer, val_dataset, max_length
)

# データローダの作成
train_dataloader = DataLoader(train_dataset_for_loader, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset_for_loader, batch_size=256)

PyTorch Lightningでファインチューニングを行う。

In [ ]:
# 9-15

class BertForMaskedLM_pl(pl.LightningModule):
    def __init__(self, model_name, lr):
        super().__init__()
        self.save_hyperparameters()
        self.model = BertForMaskedLM.from_pretrained(model_name)
    
    def training_step(self, batch, batch_idx):
        output = self.model(**batch)
        train_loss = output.loss
        self.log('train_loss', train_loss)
        return train_loss

    def validation_step(self, batch, batch_idx):
        output = self.model(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath="model/",
)

# GPUで動作させる場合、accelerator引数に'gpu'を渡す。
# 2枚以上のGPUで分散処理(DistributedDataParallel)する場合、
# devices引数に>1のintを、
# strategy引数に'ddp'(.py)や'ddp_notebook'(.ipynb)文字列を渡す
# Pytorchのプロセスtorch.cuda()がGPUメモリに残っていると動かないので注意
trainer = pl.Trainer(
    accelerator='gpu',
    devices=1, # GPUが2枚だとなぜかエラーが出る
    # strategy='ddp_notebook', # DistributedDataParallel
    max_epochs=5,
    callbacks=[checkpoint],
)

# ファインチューニング
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
model = BertForMaskedLM_pl(model_name, lr=1e-5)
trainer.fit(model, train_dataloader, val_dataloader)
best_model_path = checkpoint.best_model_path

## 9.6 性能評価

In [ ]:
# 9-16

def predict(text, tokenizer, model):
    """
    文章からBERTが予測した文章を出力
    """
    # 受け取った文章を符号化する
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = {key: value.cuda() for key, value in encoding.items()}

    # 
    with torch.no_grad():
        output = model(**encoding)
    scores = output.logits
    labels_pred = scores[0].argmax(axis=1).cpu().numpy().tolist()

    text_pred = tokenizer.convert_output_to_text(
        text, labels_pred, spans
    )
    return text_pred

適当な文章に対してBERTによる文章校正を行ってみる

In [ ]:
text_lst = [
    'ユーザーの試行に合わせた楽曲を配信する。',
    'メールに明日の会議の史料を添付した。',
    '乳酸菌で牛乳を発行するとヨーグルトができる。',
    '突然、子供が帰省を発した。'
]

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = SC_tokenizer.from_pretrained(model_name)
model = BertForMaskedLM_pl.load_from_checkpoint(best_model_path)
model = model.model.cuda()

for text in text_lst:
    text_pred = predict(text, tokenizer, model)
    print()
    print(f"入力: {text}")
    print(f"出力: {text_pred}")

次に`test_dataset`の評価を行う

In [ ]:
correct_num = 0
for sample in tqdm(test_dataset):
    pre_text = sample['pre_text']
    post_text = sample['post_text']
    text_pred = predict(pre_text, tokenizer, model)

    if post_text == text_pred:
        correct_num += 1

In [ ]:
print(f"Accuracy: {correct_num / len(test_dataset):.2f}")

誤変換の単語を特定した割合を評価する。

In [ ]:
# 9-18

correct_position_num = 0
for sample in tqdm(test_dataset):
    pre_text = sample['pre_text']
    post_text = sample['post_text']

    # 符号化
    pre_encoding = tokenizer(pre_text)
    post_encoding = tokenizer(post_text)
    pre_input_ids = pre_encoding['input_ids'] # 誤変換していた文章の符号列
    post_input_ids = post_encoding['input_ids'] # 正解文の符号列

    pre_encoding = {key: torch.tensor([value]).cuda() for key, value in pre_encoding.items()}

    # 予測
    with torch.no_grad():
        output = model(**pre_encoding)
    scores = output.logits
    pred_input_ids = scores[0].argmax(axis=1).cpu().numpy().tolist()

    # 特殊トークンを取り除く
    pre_input_ids = pre_input_ids[1:-1]
    post_input_ids = post_input_ids[1:-1]
    pred_input_ids = pred_input_ids[1:-1]

    # 誤変換した漢字を特定できているか、符号列を比較して判定
    detect_flag = True
    for pre_token, post_token, pred_token in zip(pre_input_ids, post_input_ids, pred_input_ids):
        # 正しいトークンの場合
        if pre_token == post_token:
            # 正しいトークンなのに誤って別のトークンに変換している場合
            if pre_token != pred_token:
                detect_flag = False
                break
        else:
            # 誤変換のトークンをそのままにしている場合
            if pre_token == pred_token:
                detect_flag = False
                break
    if detect_flag:
        correct_position_num += 1

In [ ]:
print(f"Accuracy: {correct_position_num / len(test_dataset):.2f}")